In [4]:
from tkinter import *
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
import unicodedata
import math
import tkinter.messagebox
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse.linalg import svds
from ast import literal_eval
def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def preProcessing(words):
    words = nltk.word_tokenize(words)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def getQueryDoc(query,tfidfdoc):
    queryDoc = {}
    for k in tfidfdoc:
        queryDoc[k] = 0;
        if k in query:
            queryDoc[k] = 1;
    return queryDoc
    
def searchQuery():
    query = queryTextBox.get()
    preproQuery = preProcessing(query)
    rank = {}
    rank['doc1'] = getCosineSimiliarity(getQueryDoc(query,tfidfDoc1.keys()),tfidfDoc1)
    rank['doc2'] = getCosineSimiliarity(getQueryDoc(query,tfidfDoc2.keys()),tfidfDoc2)
    rank['doc3'] = getCosineSimiliarity(getQueryDoc(query,tfidfDoc3.keys()),tfidfDoc3)
    rank['doc4'] = getCosineSimiliarity(getQueryDoc(query,tfidfDoc4.keys()),tfidfDoc4)
    rank['doc5'] = getCosineSimiliarity(getQueryDoc(query,tfidfDoc5.keys()),tfidfDoc5)
    tkinter.messagebox.showinfo("Related Documents",sorted(rank.items(), key=lambda x: x[1],reverse=True))
    return 

def getTermFrequency(words):
    frequency = {};
    for word in words:
        if(word in frequency):
            frequency[word] += 1
        else:
            frequency[word] = 1
    return frequency
    
def getDocumentFrequency(documentFrequency,words):
    for word in words:
        if(word in documentFrequency):
            documentFrequency[word] += 1
        else:
            documentFrequency[word] = 1
    return documentFrequency;	

def getTfIdf(documentFrequency,termFrequency):
    tfidf = {};
    for k,v in documentFrequency.items():
        tfidf[k] = 0
    for k,v in termFrequency.items():
        tfidf[k] = v * math.log(6 / ( documentFrequency[k] + 1) )
    return tfidf;

def getCosineSimiliarity(query,tfidf):
    multi = 0;
    for k,v in query.items():
        multi += v * tfidf[k]
    divide = float( (sum(query.values()) * sum(tfidf.values()) ) **0.5 )
    if divide == 0:
        return 0;
    return multi/divide;

list1 = preProcessing(open("corpus/1.txt", "r").read())
list2 = preProcessing(open("corpus/2.txt", "r").read())  
list3 = preProcessing(open("corpus/3.txt", "r").read())
list4 = preProcessing(open("corpus/4.txt", "r").read())
list5 = preProcessing(open("corpus/5.txt", "r").read())


termFrequency1 = getTermFrequency(list1)
termFrequency2 = getTermFrequency(list2)
termFrequency3 = getTermFrequency(list3)
termFrequency4 = getTermFrequency(list4)
termFrequency5 = getTermFrequency(list5)


documentFrequency = {};
documentFrequency = getDocumentFrequency(documentFrequency,list(termFrequency1.keys()))
documentFrequency = getDocumentFrequency(documentFrequency,list(termFrequency2.keys()))
documentFrequency = getDocumentFrequency(documentFrequency,list(termFrequency3.keys()))
documentFrequency = getDocumentFrequency(documentFrequency,list(termFrequency4.keys()))
documentFrequency = getDocumentFrequency(documentFrequency,list(termFrequency5.keys()))

tfidfDoc1 = getTfIdf(documentFrequency,termFrequency1)
tfidfDoc2 = getTfIdf(documentFrequency,termFrequency2)
tfidfDoc3 = getTfIdf(documentFrequency,termFrequency3)
tfidfDoc4 = getTfIdf(documentFrequency,termFrequency4)
tfidfDoc5 = getTfIdf(documentFrequency,termFrequency5)

datadict = [tfidfDoc1,tfidfDoc2,tfidfDoc3,tfidfDoc4,tfidfDoc5]
dictvectorizer = DictVectorizer(sparse=False)
features = dictvectorizer.fit_transform(datadict)

A=np.array(features)
u,s,v = svds(A,k=1)
X = u.dot(np.diag(s)).dot(v)



master = Tk() 
master.title("Mini Search Engine")
Label(master, text='Enter Query').grid(row=0) 
Button(master, text='Search',command = searchQuery, width=25).grid(row=1)
queryTextBox = Entry(master) 
searchButton = Entry(master) 
queryTextBox.grid(row=0, column=1) 
mainloop() 

